In [1]:
from datetime import datetime, timedelta
import pickle
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import re

from tldextract import extract
import langid

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 50)

from newspaper import Article

<ipython-input-1-6e87932399ef>:12: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


NOTE: Mediacloud news for state-specific collection includes news ABOUT the particular state, and as such, might sometimes contain news from national outlets where the article itself might be focused on the state itself. Most (if not all) of the collection is going to be sourced from local state-specific outlets, but not always. Currently, we are keeping all URLs within the obtained collection for the state, except those from nytimes and foxnews since those are our chosen national outlets. Trusting mediacloud's collections to be truly state-focused even when urls come from national outlets. 

In [2]:
start_date = datetime.strptime('2023-04-01', '%Y-%m-%d').date()
end_date = datetime.strptime('2023-06-30', '%Y-%m-%d').date()

In [4]:
illinois_news = pd.read_csv('data/mediacloud_illinois.csv')
print(illinois_news.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6108 entries, 0 to 6107
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ap_syndicated         6108 non-null   bool   
 1   collect_date          6108 non-null   object 
 2   feeds                 0 non-null      float64
 3   guid                  6108 non-null   object 
 4   language              5870 non-null   object 
 5   media_id              6108 non-null   int64  
 6   media_name            6108 non-null   object 
 7   media_url             6108 non-null   object 
 8   metadata              6108 non-null   object 
 9   processed_stories_id  6108 non-null   int64  
 10  publish_date          6108 non-null   object 
 11  stories_id            6108 non-null   int64  
 12  story_tags            6108 non-null   object 
 13  title                 6108 non-null   object 
 14  url                   6108 non-null   object 
 15  word_count           

In [5]:
illinois_news[['language', 'media_name', 'media_url', 'publish_date', 'title', 'url']].tail(20)

,language,media_name,media_url,publish_date,title,url
6088,en,Batavia Republican,http://www.mysuburbanlife.com/batavia/,2023-06-30 14:33:19,The Supreme Court rules for a designer who doesn’t want to make wedding websites for gay couples,https://www.shawlocal.com/news/2023/06/30/the-supreme-court-rules-for-a-designer-who-doesnt-want-to-make-wedding-websites-for-gay-couples/
6089,en,Lombard Spectator,http://www.mysuburbanlife.com/lombard/,2023-06-30 14:33:19,The Supreme Court rules for a designer who doesn’t want to make wedding websites for gay couples,https://www.shawlocal.com/news/2023/06/30/the-supreme-court-rules-for-a-designer-who-doesnt-want-to-make-wedding-websites-for-gay-couples/
6090,en,Stickney Life,http://www.mysuburbanlife.com/stickney/,2023-06-30 14:33:19,The Supreme Court rules for a designer who doesn’t want to make wedding websites for gay couples,https://www.shawlocal.com/news/2023/06/30/the-supreme-court-rules-for-a-designer-who-doesnt-want-to-make-wedding-websites-for-gay-couples/
6091,en,Oak Brook Suburban Life,http://www.mysuburbanlife.com/oakbrook/,2023-06-30 14:33:19,The Supreme Court rules for a designer who doesn’t want to make wedding websites for gay couples,https://www.shawlocal.com/news/2023/06/30/the-supreme-court-rules-for-a-designer-who-doesnt-want-to-make-wedding-websites-for-gay-couples/
6092,en,News Tribune,http://newstrib.com/,2023-06-30 14:33:19,The Supreme Court rules for a designer who doesn’t want to make wedding websites for gay couples,https://www.shawlocal.com/news/2023/06/30/the-supreme-court-rules-for-a-designer-who-doesnt-want-to-make-wedding-websites-for-gay-couples/
6093,en,midweeknews.com,http://midweeknews.com/,2023-06-30 14:33:19,The Supreme Court rules for a designer who doesn’t want to make wedding websites for gay couples,https://www.shawlocal.com/news/2023/06/30/the-supreme-court-rules-for-a-designer-who-doesnt-want-to-make-wedding-websites-for-gay-couples/
6094,en,Forest View Life,http://www.mysuburbanlife.com/forestview/,2023-06-30 14:33:19,The Supreme Court rules for a designer who doesn’t want to make wedding websites for gay couples,https://www.shawlocal.com/news/2023/06/30/the-supreme-court-rules-for-a-designer-who-doesnt-want-to-make-wedding-websites-for-gay-couples/
6095,en,Hodgkins Suburban Life,http://www.mysuburbanlife.com/hodgkins/,2023-06-30 14:33:19,The Supreme Court rules for a designer who doesn’t want to make wedding websites for gay couples,https://www.shawlocal.com/news/2023/06/30/the-supreme-court-rules-for-a-designer-who-doesnt-want-to-make-wedding-websites-for-gay-couples/
6096,en,Lake County Journal,http://www.lakecountyjournal.com/,2023-06-30 14:33:19,The Supreme Court rules for a designer who doesn’t want to make wedding websites for gay couples,https://www.shawlocal.com/news/2023/06/30/the-supreme-court-rules-for-a-designer-who-doesnt-want-to-make-wedding-websites-for-gay-couples/
6097,en,capitolfax.com,http://capitolfax.com,2023-06-30 15:51:20,Afternoon roundup,http://capitolfax.com/2023/06/30/afternoon-roundup-115/


In [13]:
relevant_news = illinois_news[~illinois_news['media_name'].isin(['Fox News', 'New York Times'])]
relevant_news = relevant_news[relevant_news['language']=='en']
relevant_news = relevant_news[['media_name', 'publish_date', 'title', 'url']]
print(relevant_news.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5849 entries, 1 to 6107
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   media_name    5849 non-null   object
 1   publish_date  5849 non-null   object
 2   title         5849 non-null   object
 3   url           5849 non-null   object
dtypes: object(4)
memory usage: 228.5+ KB
None


In [14]:
relevant_news.publish_date = pd.to_datetime(relevant_news.publish_date)
relevant_news = relevant_news[relevant_news['publish_date'].dt.date >= start_date]
relevant_news = relevant_news[relevant_news['publish_date'].dt.date <= end_date]
print(relevant_news.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5849 entries, 1 to 6107
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   media_name    5849 non-null   object        
 1   publish_date  5849 non-null   datetime64[ns]
 2   title         5849 non-null   object        
 3   url           5849 non-null   object        
dtypes: datetime64[ns](1), object(3)
memory usage: 228.5+ KB
None


In [15]:
def get_article_from_url(url):
    try:
        article = Article(url)
        article.download()
        article.parse()
        subtitle = article.meta_description
        if subtitle == '' or subtitle == ' ':
            subtitle = None
        text = article.text
        if text == '' or text == ' ':
            text = None
    except:
        subtitle = None
        text = None
    return subtitle, text

In [16]:
subtitles, texts = [], []
all_urls = list(relevant_news['url'])
for url in tqdm(all_urls):
    subtitle, text = get_article_from_url(url)
    subtitles.append(subtitle)
    texts.append(text)
relevant_news['subtitle'] = subtitles
relevant_news['text'] = texts
print(relevant_news.info())

100%|██████████| 5849/5849 [32:07<00:00,  3.03it/s]  

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5849 entries, 1 to 6107
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   media_name    5849 non-null   object        
 1   publish_date  5849 non-null   datetime64[ns]
 2   title         5849 non-null   object        
 3   url           5849 non-null   object        
 4   subtitle      1639 non-null   object        
 5   text          1738 non-null   object        
dtypes: datetime64[ns](1), object(5)
memory usage: 319.9+ KB
None


In [17]:
relevant_news.to_csv('data/illinois_article_texts_and_info.csv',
                     index=False)